# Start


This script prepares vapor pressure deficit (VPD) data from SILO and matches it with VPD from field observations for subsequent analysis.


📄 **What this script does**
1. Loads cleaned field data with topography — for example, output from `Nick_phd_data_complilation.ipynb`.
2. Downloads **SILO data** for the grid cells closest to the field sites, spanning from the first day to one day after the last day of the field observations.
3. Find the field observations with the highest and lowest temperature between 9am to 9am of each day and put them separately in `df_max_temp` and `df_min_temp`. When there are multiple observations with the highest or lowest temperatures, the ones with the lowest RH are selected.
4. Matches SILO temperature and RH values to the field observations in `df_max_temp` and `df_min_temp` based on the nearest SILO grid cell and `silo_observation_date`.
5. For both `df_max_temp` and `df_min_temp`, calculates **VPD** (vapor pressure deficit) from the matched SILO temperature and RH.
6. Saves the combined field and SILO data as `silo_max_temp.csv` and `silo_min_temp.csv` in the `output/csv` folder.


⚠️ **Important notes**
* Before running the script, set all variables in the **first cell**, and delete the **second cell** if not using a Google Colab environment.  
  *(The script was developed for use in Google Colab and has not been tested outside of it.)*
* The **first and last observation dates** of each site are **disregarded** because there is no temperature data for the whole day for those dates.
* Obsevations with timestamps between 8.59 am of 1 April 2020 to 9.00 am of 2 April 202 have `silo_observation_date` as 2 April 2020.


In [ ]:
input_file_name = 'in-situ_topography_phd.csv'
max_temp_output_file_name = 'silo_max_temp_vpd_phd.csv'
min_temp_output_file_name = 'silo_min_temp_vpd_phd.csv'

download_silo_data = True

In [ ]:
import sys
sys.path.append('../..')
from Utils.vpd import calculate_vpd

# Loading in-situ and remote data


In [ ]:
# Load in-situ_topography.csv as the main df

import pandas as pd
import os

df = pd.read_csv(os.path.join("..", "..", "output", "csv", input_file_name))
df['Datetime'] = pd.to_datetime(df['Datetime'])

first_date, day_after_last_date = min(df['Datetime']).strftime("%Y%m%d"), (max(df['Datetime']) + pd.Timedelta(days=1)).strftime("%Y%m%d")
print("First date: ", first_date, ", One day after the last date: ", day_after_last_date)

df.head()

In [ ]:
# Load silo data

def nearest_silo_cell(x, y):
    return (round(round(x / 0.05) * 0.05, 3), round(round(y / 0.05) * 0.05, 3))

# Make a list of locations of all sites and reduce it to silo grid cell locations
df['silo_X'] = df.apply(lambda row: nearest_silo_cell(row['X'], row['Y'])[0], axis=1)
df['silo_Y'] = df.apply(lambda row: nearest_silo_cell(row['X'], row['Y'])[1], axis=1)
silo_cell_locations_list = list(set((x, y) for x, y in df[['silo_X', 'silo_Y']].values))
print("silo_cell_locations_list: ", silo_cell_locations_list)

# Use Python loop and curl to download all silo data
silo_data_dir = os.path.join("..", "..", "Data", "silo")
if download_silo_data:
    os.makedirs(silo_data_dir, exist_ok=True)
    for x, y in silo_cell_locations_list:
        output_file_path = os.path.join(silo_data_dir, f"silo_data_{x}_{y}.csv")
        url = f"https://www.longpaddock.qld.gov.au/cgi-bin/silo/DataDrillDataset.php?lat={y}&lon={x}&format=csv&start={first_date}&finish={day_after_last_date}&username=noemail@net.com&dataset=Official&comment=xnhg"
        print(f"Downloading {url} to {output_file_path}")
        !curl -L "{url}" -o "{output_file_path}" -C -

# Combining in-situ and remote data into a single dataframe

In [ ]:
# Find in-situ data's hottest and coldest observation of each day

# Assign silo_observation_date to each observation
mask = (df['Datetime'].dt.hour > 9) | ((df['Datetime'].dt.hour == 9) & (df['Datetime'].dt.minute > 0))
df.loc[mask, 'silo_observation_date'] = df.loc[mask, 'Datetime'] + pd.Timedelta(days=1)
df.loc[~mask, 'silo_observation_date'] = df.loc[~mask, 'Datetime']
df['silo_observation_date'] = pd.to_datetime(df['silo_observation_date']).dt.date

# Remove rows of the first and last dates of each site because those days lack some hours
first_last_dates = df.groupby('SiteID')['silo_observation_date'].agg(['min', 'max']).reset_index()
df = df.merge(first_last_dates, on='SiteID', how='left')
df_date_filtered = df[(df['silo_observation_date'] != df['min']) & (df['silo_observation_date'] != df['max'])]
df_date_filtered = df_date_filtered.drop(columns=['min', 'max'])

# Select only rows with max tempurature of the day
df_date_filtered['Day_max_temp'] = df_date_filtered.groupby(['SiteID', 'silo_observation_date'])['Temperature'].transform('max')
df_max_temp = df_date_filtered[df_date_filtered['Temperature'] == df_date_filtered['Day_max_temp']]
print("In-situ day maximum temperature observations:")
print(df_max_temp.head())
print('\n')

# Select only rows with min tempurature of the day
df_date_filtered['Day_min_temp'] = df_date_filtered.groupby(['SiteID', 'silo_observation_date'])['Temperature'].transform('min')
df_min_temp = df_date_filtered[df_date_filtered['Temperature'] == df_date_filtered['Day_min_temp']]
print("In-situ day minimum temperature observations:")
df_min_temp.head()

# -------------------------------------- Investigation code below --------------------------------------
# df_max_temp[df_max_temp['Datetime'].dt.hour < 9]

# import datetime
# df[(df['SiteID'] == 79) & (df['Date'] == datetime.date(2019, 1, 30))]
# df[(df['SiteID'] == 90) & (df['Date'] == datetime.date(2019, 2, 19))]  ## Hottest in the morning
# df[(df['SiteID'] == 90) & (df['Date'] == datetime.date(2019, 3, 6))]
# df[(df['SiteID'] == 142) & (df['Date'] == datetime.date(2019, 3, 27))]
# df[(df['SiteID'] == 78) & (df['Date'] == datetime.date(2019, 1, 19))]

# df_date_filtered.groupby(['SiteID', 'Date'])['Datetime'].apply(lambda x: x.nunique())

In [ ]:
# Use only one observation per sit per day

df_max_temp = df_max_temp.loc[
    df_max_temp.groupby(['SiteID', 'silo_observation_date'])['RH'].idxmin()
]
df_min_temp = df_min_temp.loc[
    df_min_temp.groupby(['SiteID', 'silo_observation_date'])['RH'].idxmin()
]
df_max_temp.head()

# Investigating the number of obs per site per day
# df_max_temp.groupby(['SiteID', 'silo_observation_date'])['Datetime'].apply(lambda x: x.nunique())
# Check the accuracy of the code
# import datetime
# df_max_temp[(df_max_temp['SiteID'] == 264) & (df_max_temp['silo_observation_date'] == datetime.date(2020, 3, 21))]

In [ ]:
# Fill in df with silo data at the hottest time of the day (max_temp, RH_tmax)

# For each row, open silo data csv file one-by-one to get data
def get_silo_value(row, column):
    file_path = os.path.join(silo_data_dir, f"silo_data_{row['silo_X']}_{row['silo_Y']}.csv")
    df_silo = pd.read_csv(file_path)

    df_silo['Datetime'] = pd.to_datetime(df_silo['YYYY-MM-DD'], format='%Y-%m-%d')
    target_date = row['silo_observation_date']
    silo_value = df_silo.loc[df_silo['Datetime'].dt.date == target_date, column].values[0]

    return silo_value

df_max_temp['silo_Temperature'] = df_max_temp.apply(lambda row: get_silo_value(row, 'max_temp'), axis=1)
df_max_temp['silo_RH'] = df_max_temp.apply(lambda row: get_silo_value(row, 'rh_tmax'), axis=1)
df_max_temp.head()

In [ ]:
# Fill in df with silo data at the coldest time of the day (min_temp, RH_tmin)

df_min_temp['silo_Temperature'] = df_min_temp.apply(lambda row: get_silo_value(row, 'min_temp'), axis=1)
df_min_temp['silo_RH'] = df_min_temp.apply(lambda row: get_silo_value(row, 'rh_tmin'), axis=1)
df_min_temp.head()

In [ ]:
# Investigate null values. Site 251 is located in the ocean.
print("df_max_temp length: ", len(df_max_temp))
print("df_min_temp length: ", len(df_min_temp))
df_max_temp[(df_max_temp.isna().any(axis=1)) & (df_max_temp['SiteID'] != 251)]
df_min_temp[(df_min_temp.isna().any(axis=1)) & (df_min_temp['SiteID'] != 251)]

# Calculating remote VPD from remote temperature and remote relative humidity

In [ ]:
# Write a function for deriving VPD from temp and RH

import math
import numpy as np

def calculate_vpd(temp, rh):
    if pd.isna(temp) or pd.isna(rh):
        return np.nan
    es = 0.6108 * math.exp(17.27 * temp / (237.3 + temp))
    e = es * rh / 100
    vpd = es - e
    return vpd

df_max_temp['silo_VPD'] = df_max_temp.apply(lambda row: calculate_vpd(row['silo_Temperature'], row['silo_RH']), axis=1)
df_min_temp['silo_VPD'] = df_min_temp.apply(lambda row: calculate_vpd(row['silo_Temperature'], row['silo_RH']), axis=1)
df_max_temp.head()

# Save the resulting dataframes

In [ ]:
# Drop helper columns
df_max_temp.drop(columns=['Day_max_temp'], inplace=True)
df_min_temp.drop(columns=['Day_max_temp', 'Day_min_temp'], inplace=True)

In [ ]:
# save df
df_max_temp.to_csv(os.path.join("..", "..", "output", "csv", max_temp_output_file_name), index=False)
df_min_temp.to_csv(os.path.join("..", "..", "output", "csv", min_temp_output_file_name), index=False)